<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Voronoi_in_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Approximate Voronoi Diagram in C

This code was written by Isaiah Hendrick.

First, a sequential implementation in C.

In [ ]:
%%writefile voronoi.c
// compile with "gcc voronoi.c -o voronoi -lm"
// run with "./voronoi"

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define SEED_ROW_SIZE   4
#define SEED_COL_SIZE   2

void calculate_voronoi(int, double *, double *);

int main(int argc, char *argv[]) {
  int size = 4096;
  double(*area)[size] = calloc(size, sizeof *area);

  double seed_vals[SEED_ROW_SIZE][SEED_COL_SIZE] = {
      {0, 0},
      {0, size - 1},
      {size - 1, size - 1},
      {size - 1, 0}
  };

  double *seeds = (double *)malloc(sizeof(seed_vals));
  memcpy(seeds, seed_vals, sizeof(seed_vals));

  clock_t start = clock();
  calculate_voronoi(size, (double *)seeds, (double *)area);
  clock_t end = clock();
  double time_taken = (double)(end - start) / CLOCKS_PER_SEC;

  printf("SEQUENTIAL VORONOI IMPLEMENTATION\n");
  printf("---------------------------------\n");
  printf("Array size: %d\n", size);
  printf("Time taken: %f s\n", time_taken);

  free(seeds);
  free(area);

  return 0;
}


void calculate_voronoi(int size, double *pSeeds, double *pArea) {
  double closest_seed, closest_distance, dist;

  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      closest_seed = -1;
      closest_distance = pow(size, 2);

      for (int k = 0; k < SEED_ROW_SIZE; k++) {
        dist = sqrt(pow(pSeeds[k * SEED_COL_SIZE + 0] - i, 2) +
                    pow(pSeeds[k * SEED_COL_SIZE + 1] - j, 2));

        if (dist < closest_distance) {
          closest_distance = dist;
          closest_seed = k;
        }
      }

      pArea[i * size + j] = closest_seed;
    }
  }
}


Overwriting voronoi.c


Now, the compilation:

In [ ]:
!gcc voronoi.c -o voronoi -O3 -lm

And the execution and timing:

In [ ]:
!time ./voronoi

SEQUENTIAL VORONOI IMPLEMENTATION
---------------------------------
Array size: 4096
Time taken: 0.234333 s

real	0m0.242s
user	0m0.188s
sys	0m0.053s


Now with OpenMP:


In [ ]:
%%writefile voronoi_openmp.c

//
// compile with "gcc voronoi_openmp.c -o voronoi_openmp -lm -fopenmp"
// run with "OMP_NUM_THREADS=2 ./voronoi_openmp"

#include <math.h>
#include <omp.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define SEED_ROW_SIZE 4
#define SEED_COL_SIZE 2

void calculate_voronoi(int, double *, double *);
int gen_voronoi_graph(int, double *);

int main(int argc, char *argv[]) {
  int size = 4096;
  double(*area)[size] = calloc(size, sizeof *area);

  double seed_vals[SEED_ROW_SIZE][SEED_COL_SIZE] = {
      {0, 0}, {0, size - 1}, {size - 1, size - 1}, {size - 1, 0}};

  double *seeds = (double *)malloc(sizeof(seed_vals));
  memcpy(seeds, seed_vals, sizeof(seed_vals));

  double start = omp_get_wtime();
  calculate_voronoi(size, (double *)seeds, (double *)area);
  double end = omp_get_wtime();

  printf("PARALLEL VORONOI IMPLEMENTATION\n");
  printf("-------------------------------\n");
  printf("Array size: %d\n", size);
  printf("Time taken: %f s\n", end - start);

  free(seeds);
  free(area);


}

void calculate_voronoi(int size, double *pSeeds, double *pArea) {
  double closest_seed, closest_distance, dist;

  #pragma omp parallel for collapse(2) private(closest_seed, closest_distance, dist)
  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      closest_seed = -1;
      closest_distance = pow(size, 2);

      for (int k = 0; k < SEED_ROW_SIZE; k++) {
        dist = sqrt(pow(pSeeds[k * SEED_COL_SIZE + 0] - i, 2) +
                    pow(pSeeds[k * SEED_COL_SIZE + 1] - j, 2));

        if (dist < closest_distance) {
          closest_distance = dist;
          closest_seed = k;
        }
      }

      pArea[i * size + j] = closest_seed;
    }
  }
}




Overwriting voronoi_openmp.c


Now the compilation with OpenMP

In [ ]:
!gcc voronoi_openmp.c -o voronoi_openmp -fopenmp -O3 -lm

And now the execution:

In [ ]:
!time OMP_NUM_THREADS=2 ./voronoi_openmp

PARALLEL VORONOI IMPLEMENTATION
-------------------------------
Array size: 4096
Time taken: 0.201455 s

real	0m0.211s
user	0m0.351s
sys	0m0.054s
